# Speeding Up Your Python Codes 1000x

## Introduction

Welcome to "Speeding Up Your Python Codes 1000x"!
In the next 45 minutes, we will learn tricks to unlock python performance and increase a sample code's speed by a factor of 1000x!

## The $n$-body Problem and the Leapfrog Algorithm

The $n$-body problem involves simulating the motion of many bodies interacting with each other through (usually) gravitational forces.
Each body experiences a force from every other body, making the simulation computationally intensive---ideal for exploring optimization techniques.

The leapfrog algorithm is a simple, robust numerical integrator that alternates between updating velocities ("kicks") and positions ("drifts").
It is popular because it conserves energy and momentum well over long simulation periods.

### Gravitational Force/Acceleration Equation

For any two bodies labeled by $i$ and $j$, the direct gravitational force exerted on body $i$ by body $j$ is given by:
\begin{align}
  \mathbf{f}_{ij} = -G m_i m_j\frac{\mathbf{r}_i - \mathbf{r}_j\ \ \ }{|\mathbf{r}_i - \mathbf{r}_j|^{3/2}},
\end{align}
where:
* $G$ is the gravitational constant.
* $m_i$ and $m_j$ are the masses of the bodies.
* $\mathbf{r}_i$ and $\mathbf{r}_j$ are their position vectors.

Summing the contributions from all other bodies gives the net force (and thus the acceleration) on body $i$:
\begin{align}
  \mathbf{f}_i 
  = \sum_{j\ne i} \mathbf{f}_{ij} 
  = -G m_i \sum_{j\ne i} m_j \frac{\mathbf{r}_i - \mathbf{r}_j\ \ \ }{|\mathbf{r}_i - \mathbf{r}_j|^3},
\end{align}

Given Newton's law $f = m a$, the "acceleration" applied on body $i$ caused by all other bodies is:
\begin{align}
  \mathbf{a}_i 
  = \sum_{j\ne i} \mathbf{a}_{ij} 
  = -G \sum_{j\ne i} m_j \frac{\mathbf{r}_i - \mathbf{r}_j\ \ \ }{|\mathbf{r}_i - \mathbf{r}_j|^3}.
\end{align}

Choosing the right units so $G = 1$.
Here is a pure Python implementation of the gravitational acceleration:

In [ ]:
def acc1(m, r):
    
    n = len(m)
    A = []
    
    for i in range(n):
        ax_i, ay_i, az_i = 0, 0, 0
        for j in range(n):
            if j != i:
                xi, yi, zi = r[i]
                xj, yj, zj = r[j]

                ax_ij = - m[j] * (xi - xj) / ((xi - xj)**2 + (yi - yj)**2 + (zi - zj)**2)**(3/2)
                ay_ij = - m[j] * (yi - yj) / ((xi - xj)**2 + (yi - yj)**2 + (zi - zj)**2)**(3/2)
                az_ij = - m[j] * (zi - zj) / ((xi - xj)**2 + (yi - yj)**2 + (zi - zj)**2)**(3/2)

                ax_i += ax_ij
                ay_i += ay_ij
                az_i += az_ij

        A.append((ax_i, ay_i, az_i))

    return A

We may try using this function to evaluate the gravitational force between two particles, with with mass $m = 1$, at a distance of $2$.
We expect the "acceleration" be -1/4 in the direction of their seperation.

In [ ]:
m = [1.0, 1.0]
r = [
    (+1.0, 0.0, 0.0),
    (-1.0, 0.0, 0.0),
]
a_ref = [
    (-0.25, 0.0, 0.0),
    (+0.25, 0.0, 0.0),
]

In [ ]:
a = acc1(m, r)

In [ ]:
a

In [ ]:
assert acc1(m, r) == a_ref